<a href="https://colab.research.google.com/github/theroyakash/MonkeyNet/blob/master/MonkeyNetPyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "theroyakash"
os.environ['KAGGLE_KEY'] = "👀"
!kaggle datasets download -d slothkong/10-monkey-species

100% 545M/547M [00:20<00:00, 32.7MB/s]
100% 547M/547M [00:20<00:00, 28.1MB/s]


In [ ]:
!unzip /content/10-monkey-species.zip

In [23]:
import torch
import torch.nn as nn
from PIL import Image
import torch.utils.data as data
import torchvision
from torchvision import transforms, models
from tqdm.auto import tqdm

In [4]:
import numpy as np
from matplotlib import pyplot as plt

In [5]:
# Set seed for recreating results
import random

torch.manual_seed(123)
np.random.seed(123)
random.seed(123)

In [6]:
label2name = {
    'n0':'alouatta_palliata',
    'n1':'erythrocebus_patas',
    'n2':'cacajao_calvus',
    'n3':'macaca_fuscata',
    'n4':'cebuella_pygmea',
    'n5':'cebus_capucinus',
    'n6':'mico_argentatus',
    'n7':'saimiri_sciureus',
    'n8':'aotus_nigriceps',
    'n9':'trachypithecus_johnii',
}

name2id = {
    'alouatta_palliata':0,
    'erythrocebus_patas':1,
    'cacajao_calvus':2,
    'macaca_fuscata':3,
    'cebuella_pygmea':4,
    'cebus_capucinus':5,
    'mico_argentatus':6,
    'saimiri_sciureus':7,
    'aotus_nigriceps':8,
    'trachypithecus_johnii':9,
}

In [7]:
class Transform():
    """
    Image Transformer class.

        Args:
            - ``resize``: Image size after resizing
            - ``mean``: R,G,B avg of each channel
            - ``std``: Standard deviation of each channel
    """

    def __init__(self, resize, mean, std):
        self.transform = {
            'train': transforms.Compose([
                                         transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor(),
                                         transforms.Normalize(mean, std=std)
            ]),
            'validation': transforms.Compose([
                                              transforms.Resize(resize),
                                              transforms.CenterCrop(resize),
                                              transforms.ToTensor(),
                                              transforms.Normalize(mean, std)
            ])
        }

    def __call__(self, image, phase='train'):
        return self.transform[str(phase)](image)

In [8]:
import glob

def pathlist(phase):
    root = '/content/'
    target = os.path.join(root+phase+'/**/**/*.jpg')
    paths = []

    for path in glob.glob(target):
        paths.append(path)
    
    return paths

In [9]:
train_list = pathlist(phase='training')
validation_list = pathlist(phase='validation')

In [25]:
class Dataset(torch.utils.data.Dataset):
    """
    Dataset class.

        Args:
            - ``path_list``: List of paths for the data
            - ``transform``: Image Transform Object
            - ``phase``: Traning or Validation
    """
    
    def __init__(self, path_list, transform, phase):
        self.path_list = path_list
        self.transform = transform
        self.phase = phase
    
    def __len__(self):
        return len(self.path_list)
    
    def __getitem__(self, idx):
        image_path = self.path_list[idx]
        image = Image.open(image_path)

        # Pre-processing:
        transformed_image = self.transform(image, self.phase)

        # Get Label of the Image
        array = image_path.split('/')
        label = array[-2]
        name = label2name[label]

        # Transform Label to Number
        label_number = name2id[name]

        return transformed_image, label_number

In [26]:
image_size = 224
mean = (0.485,0.456,0.406)
std = (0.229,0.224,0.225)

In [27]:
# Training and Validation Dataset Generator from Dataset Class
training_dataset = Dataset(path_list=train_list, transform=Transform(image_size, mean, std), phase='train')
validation_dataset = Dataset(validation_list, transform=Transform(image_size, mean, std), phase='validation')

In [28]:
batch_size = 64
training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)

In [29]:
dataloaders = {
    'train': training_dataloader,
    'validation': validation_dataloader
}

In [30]:
network = torchvision.models.vgg16(pretrained=True)
network.classifier[6] = nn.Linear(in_features=4096, out_features=10)

In [31]:
network.train()
criterion = nn.CrossEntropyLoss()

# Fine tuning
param_to_update_1 = []
param_to_update_2 = []
param_to_update_3 = []

# learn parameter list
update_param_names_1 = ['features']
update_param_names_2 = ['classifier.0.weight','classifier.0.bias','classifier.3.weight','classifier.3.bias']
update_param_names_3 = ['classifier.6.weight','classifier.6.bias']

In [32]:
for name,param in network.named_parameters():
    print(name)
    if update_param_names_1[0] in name:
        param.requires_grad = True
        param_to_update_1.append(param)
    elif name in update_param_names_2:
        param.requires_grad = True
        param_to_update_2.append(param)
    elif name in update_param_names_3:
        param.requires_grad = True
        param_to_update_3.append(param)
    else:
        param.requires_grad = False

features.0.weight
features.0.bias
features.2.weight
features.2.bias
features.5.weight
features.5.bias
features.7.weight
features.7.bias
features.10.weight
features.10.bias
features.12.weight
features.12.bias
features.14.weight
features.14.bias
features.17.weight
features.17.bias
features.19.weight
features.19.bias
features.21.weight
features.21.bias
features.24.weight
features.24.bias
features.26.weight
features.26.bias
features.28.weight
features.28.bias
classifier.0.weight
classifier.0.bias
classifier.3.weight
classifier.3.bias
classifier.6.weight
classifier.6.bias


In [33]:
optimizer = torch.optim.SGD([
    {'params':param_to_update_1,'lr':1e-4},
    {'params':param_to_update_2,'lr':5e-4},
    {'params':param_to_update_3,'lr':1e-3},
    ],momentum=0.9)

In [34]:
def train_model(net,dataloaders_dict,criterion,optimizer,num_epochs):

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device} for deep learning')
    network.to(device)

    history_train_loss = []
    history_train_acc = []
    history_val_loss = []
    history_val_acc = []

    torch.backends.cudnn.benchmark = True
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1,num_epochs))
        print('-------------------------------------')

        for phase in ['train','validation']:
            if phase == 'train':
                network.train()
            else:
                network.eval()
            
            epoch_loss = 0.0
            epoch_corrects = 0

            # pick mini batch from dataloader
            for inputs,labels in tqdm(dataloaders_dict[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                # init optimizer
                optimizer.zero_grad()
                # calculate forward
                with torch.set_grad_enabled(phase=='train'):
                    outputs = network(inputs)
                    loss = criterion(outputs,labels) # calculate loss
                    _,preds = torch.max(outputs,1) # predict label

                    # backward (train only)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                    # update loss sum
                    epoch_loss += loss.item() * inputs.size(0)
                    # correct answer count 
                    epoch_corrects += torch.sum(preds == labels.data)
            # show loss and correct answer rate per epoch 
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            if phase == 'train':
                history_train_loss.append(epoch_loss)
                history_train_acc.append(epoch_acc)
            else:
                history_val_loss.append(epoch_loss)
                history_val_acc.append(epoch_acc)
    return history_train_loss,history_train_acc,history_val_loss,history_val_acc

In [35]:
num_epochs=10
train_loss,train_acc,val_loss,val_acc = train_model(network, 
                                                    dataloaders, 
                                                    criterion,
                                                    optimizer, 
                                                    num_epochs=num_epochs)

Using device: cuda for deep learning
Epoch 1/10
-------------------------------------



train Loss: 1.3301 Acc: 0.5794



validation Loss: 0.2261 Acc: 0.9743
Epoch 2/10
-------------------------------------



train Loss: 0.1631 Acc: 0.9653



validation Loss: 0.0503 Acc: 0.9963
Epoch 3/10
-------------------------------------


...


validation Loss: 0.0166 Acc: 1.0000
Epoch 9/10
-------------------------------------



train Loss: 0.0274 Acc: 0.9936



validation Loss: 0.0164 Acc: 1.0000
Epoch 10/10
-------------------------------------



train Loss: 0.0370 Acc: 0.9927



validation Loss: 0.0150 Acc: 1.0000
